## PhaseObjC - DEW Quartz Solubility

Required python code to load the PhaseObjC library.  The library libphaseobjc.dylib (see build instructions in README.md) must be locatable to teh system in a standard location (by default /usr/local/lib)

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

#### Create a python reference to the DEWFluid class and instantiate an instance of that class.  
In Objective-C the code has the form:
```
obj = [[CpxBerman alloc] init]
```
and in python:

In [ ]:
DEWFluid = ObjCClass('DEWFluid')
obj = DEWFluid.alloc().init()

#### Create a python reference to the Quartz class and instantiate the phase

In [ ]:
Quartz = ObjCClass('QuartzBerman')
qtz = Quartz.alloc().init()

##### Obtain properties of the phase inherited from the PhaseBase class.  

In [ ]:
print (obj.phaseName)
print (qtz.phaseName)

## Solution Protocol Functions

### Solution component and species number ...
--> Retrieves the number of endmember components in the system
```
(NSUInteger)numberOfSolutionComponents  
```
--> Retrieves the number of species (dependent endmembers with positive mole fractions) in the soluton
```
(NSUInteger)numberOfSolutionSpecies
```
Note that the number of components (nc) may be the same as the number of endmember species (ns) if the solution does not involve speciation (complexing) or if the solid solution is not a reciprocal solution.

In [ ]:
nc = obj.numberOfSolutionComponents()
print ('Number of components = ', nc)
ns = obj.numberOfSolutionSpecies()
print ('Number of species = ', ns)

### Information about solution components ...
Return name, formula and molecular weight of each end-member component in the solution.  Note teh use of the PhaseBase class (for further info see the Stoichiometric Phase notebook examples)  

--> Retrieves superclass instance of PhaseBase object for component at specified index  
```
(id)componentAtIndex:(NSUInteger)index  
```

In [ ]:
PhaseBase = ObjCClass('PhaseBase')
print ('Component name'.ljust(20), '\t', 'Formula'.ljust(20), '\t', 'Molecular weight (g/mol)')
for i in range(0, nc):
    component = obj.componentAtIndex_(i)
    print (component.phaseName.ljust(20), '\t', component.phaseFormula.ljust(20), '\t', component.mw)

### Create a vector of moles of endmember components ...
Allocate a "c"-type pointer to a double precision one-dimensional array, and initialize the array to hold the moles of each component in the solution

In [ ]:
import ctypes
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
m[ 0] =  55.55
m[ 1] =   0.0
m[ 2] =   0.0
m[ 3] =   0.0
m[ 4] =   0.0
m[ 5] =   0.0
m[ 6] =   0.0
m[ 7] =   0.04223 # 0.03 at 400 C and 2 kb, 0.3 at 600 C and 10 kb
m[ 8] =   0.0
m[ 9] =   0.0
m[10] =   0.0
m[11] =   0.0
m[12] =   0.0
m[13] =   0.0
m[14] =   0.0
m[15] =   0.0
m[16] =   0.0
for i in range (0, nc):
    component = obj.componentAtIndex_(i)
    if m[i] > 0.0:
        print ('moles of (', component.phaseName.ljust(20), ') = ', m[i])

### Test the mole vector and output derived quantities ...


In [ ]:
if (obj.testPermissibleValuesOfComponents_(m) == 1):
    print ('mole input is feasible')
else:
    print ('mole input is infeasible')
    
print ('Total moles = ', obj.totalMolesFromMolesOfComponents_(m))

mole_frac_pointer = obj.convertMolesToMoleFractions_(m)
print ('component name'.ljust(20), 'component formula'.ljust(20), 'mole fraction')
for i in range (0, nc):
    if m[i] > 0.0:
        print (obj.componentAtIndex_(i).phaseName.ljust(20), obj.componentAtIndex_(i).phaseFormula.ljust(20), mole_frac_pointer.valueAtIndex_(i))

moles_pointer = obj.convertMolesToElements_(m)
ne = moles_pointer.size
formula = ''
for i in range(0, ne):
    value = moles_pointer.valueAtIndex_(i)
    if value != 0.0:
        name = PhaseBase.elementNameFromAtomicNumber_(i)
        formula = formula + name + '(' + str(value) + ')'
print ('Solution formula = ', formula)

#### Change convention for Berman properties
Use Helgeson (SUPCRT) convention of Gibbs free energy of formation rather than enthalpy of formation at Tr, Pr  
Do not implement quartz enthalpy correction that is used in rhyolite-MELTS

In [ ]:
Quartz.enableGibbsFreeEnergyReferenceStateUsed()
Quartz.disableQuartzCorrectionUsed()

### Compute activties and chemical potentials of endmember components/species ...

In [ ]:
BermanGTrPr = -856288.0
t = 673.15
p = 1000.0

activity = obj.getActivityFromMolesOfComponents_andT_andP_(m, t, p)
potential = obj.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
print ('Gibbs free energy (J) = ', obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m, t, p))
print ('component, activity, chemical potential, mu0')

mu0_quartz = qtz.getGibbsFreeEnergyFromT_andP_(t, p)

for i in range (0, nc):
    if m[i] > 0.0:
        component = obj.componentAtIndex_(i)
        g = component.getGibbsFreeEnergyFromT_andP_(t, p)
        print ("{0:>20s} {1:10.6f} {2:15.2f} {3:15.2f}".format(component.phaseName, activity.valueAtIndex_(i), potential.valueAtIndex_(i), g))
print ("{0:>20s} {1:15.2f}".format('Quartz', mu0_quartz))
import numpy as np
muSpecies = obj.chemicalPotentialsOfSpeciesFromMolesOfComponents_andT_andP_(m, t, p)
print ('species, activity, mu, mu0')
for i in range (0, ns):
    if muSpecies.valueAtIndex_(i) != 0.0:
        pure = obj.componentAtIndex_(i)
        g = pure.getGibbsFreeEnergyFromT_andP_(t, p)
        print("{0:>20s} {1:10.6e} {2:15.2f} {3:15.2f}".format(obj.nameOfSolutionSpeciesAtIndex_(i), np.exp((muSpecies.valueAtIndex_(i)-g)/8.3143/t), muSpecies.valueAtIndex_(i), g))
print ('deltaG quartz -> SiO2 (aq)', muSpecies.valueAtIndex_(7)-mu0_quartz)
print ("at log (molality) = {0:10.6e}".format(np.log10(m[7])))

In [ ]:
print ("{0:>10s} {1:10.6f} {2:10.6f} {3:10.6f}".format('G TP TPr TrPr', qtz.getGibbsFreeEnergyFromT_andP_(t, p), qtz.getGibbsFreeEnergyFromT_andP_(t, 1.0), qtz.getGibbsFreeEnergyFromT_andP_(298.15, 1.0)))
print ("{0:>10s} {1:10.6f} {2:10.6f} {3:10.6f}".format('H TP TPr TrPr', qtz.getEnthalpyFromT_andP_(t, p), qtz.getEnthalpyFromT_andP_(t, 1.0), qtz.getEnthalpyFromT_andP_(298.15, 1.0)))
print ("{0:>10s} {1:10.6f} {2:10.6f} {3:10.6f}".format('S TP TPr TrPr', qtz.getEntropyFromT_andP_(t, p), qtz.getEntropyFromT_andP_(t, 1.0), qtz.getEntropyFromT_andP_(298.15, 1.0)))
print ("{0:>10s} {1:10.6f} {2:10.6f} {3:10.6f}".format('C TP TPr TrPr', qtz.getHeatCapacityFromT_andP_(t, p), qtz.getHeatCapacityFromT_andP_(t, 1.0), qtz.getHeatCapacityFromT_andP_(298.15, 1.0)))

HTP = qtz.getEnthalpyFromT_andP_(t, p)
HTrPr = qtz.getEnthalpyFromT_andP_(298.15, 1.0)
STP = qtz.getEntropyFromT_andP_(t,p)
STrPr = qtz.getEntropyFromT_andP_(298.15, 1.0)
GTP = qtz.getGibbsFreeEnergyFromT_andP_(t, p)
GTrPr = qtz.getGibbsFreeEnergyFromT_andP_(298.15, 1.0)

print ('G(T,P)-G(Tr,Pr) = ', GTP - GTrPr)

BermanG = -856288
BermanH = -910700

print ('Berman deltaG-deltaH formation at 298 K and 1 bar', BermanG - BermanH)
print ('BermanG + integral from PhaseObjC', BermanG+GTP-GTrPr)

print ('Dimitri at 400C and 1KB deltaG quartz = ', -210026.5*4.184)
print ('Dimitri at 400C and 1KB deltaG SiO2,aq = ', -204755.13*4.184)
print ('Dimitri at 400C and 1KB deltaG Si2O4 = ', -413054.54*4.184)